# RAVE Latent Space Exploration

In [1]:
import librosa
import torch
import torchaudio
import IPython.display as ipd

In [2]:
tapped_dict = {
    "ac_dc": "../data/tapped/ac_dc-back_in_black-tapped.flac",
    "amy_winehouse": "../data/tapped/Tears Dry On Their Own - Amy Whinehouse-tapped.flac",
    "amália": "../data/tapped/Barco Negro (Mão Preta) - Amália Rodrigues-tapped.flac",
    "arctic_monkeys": "../data/tapped/arctic_monkeys-the_view_from_the_afternoon-tapped.flac",
    "beatles1": "../data/tapped/Come Together - The Beatles-tapped.flac",
    "beatles2": "../data/tapped/the_beatles-in_my_life-tapped.flac",
    "capitão_fausto": "../data/tapped/Morro na Praia - Capitão Fausto-tapped.flac",
    "david_miguel": "../data/tapped/Inatel - David e Miguel-tapped.flac",
    "gorillaz": "../data/tapped/Clint Eastwood - Gorillaz-tapped.flac",
    "ibeyi": "../data/tapped/River - Ibeyi-tapped.flac",
    "linda_martini": "../data/tapped/Febril - Linda Martini-tapped.flac",
    "michael_jackson": "../data/tapped/Billie Jean - Michael Jackson-tapped.flac",
    "qotsa": "../data/tapped/qotsa_no_one_knows-tapped.flac",
    "queen": "../data/tapped/queen-another_one_bites_the_dust-tapped.flac",
    "slowj": "../data/tapped/slowj-mundança-tapped.flac",
    "strokes": "../data/tapped/Reptilia - The Strokes-tapped.flac",
    "white_stripes": "../data/tapped/white_stripes-seven_nation_army-tapped.flac"
}

In [3]:
def read_audio(file_path, trim_interval=None, mono=True, print_it=False):
    audio, sr = librosa.load(file_path, mono=mono)
    audio_dim = len(audio.shape)
    if not mono and audio_dim == 1:
        audio = np.asarray((audio, audio))
    if trim_interval is not None:
        ti = trim_interval[0]
        tf = trim_interval[1]
        audio = trim_audio(audio, sr, ti, tf, mono)
    if print_it:
        print(audio.shape)
        print(sr)
    return audio, sr

def trim_audio(audio, sr, ti, tf, mono=True):
    i = ti * sr
    f = tf * sr
    if mono:
        t_audio = audio[i:f]
    else:
        t_audio = audio[:, i:f]
    return t_audio

In [4]:
#song = "ac_dc"
#song = "amália"
#song = "amy_winehouse"
#song = "arctic_monkeys"
#song = "beatles1"
#song = "beatles2"
##song = "capitão_fausto"
#song = "david_miguel"
#song = "gorillaz"
#song = "ibeyi"
#song = "linda_martini"
#song = "michael_jackson"
#song = "qotsa"
##song = "queen"
#song = "slowj"
song = "strokes"
#song = "white_stripes"

trim = True

trim_interval = None
# choose the interval in which you want to trim the audio (in seconds)
if trim:
    trim_interval = (115, 122)


In [5]:
tapped_path = tapped_dict[song]

tapped_audio, sr = read_audio(tapped_path, trim_interval)

In [11]:
model_name = 'GMDrums_v3_29-09_3M_streaming'
model_name = 'percussion'
model = torch.jit.load(f'../models/{model_name}.ts')

x = torch.from_numpy(tapped_audio).reshape(1,1,-1)

# encode and decode the audio with RAVE
z = model.encode(x)

x_hat = model.decode(z)
waveform_tensor = torch.squeeze(x_hat, 0)

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__.py", line 92, in encode
    x4 = (pqmf).forward(x3, )
    encoder = self.encoder
    mean, scale, = (encoder).forward(x4, )
                    ~~~~~~~~~~~~~~~~ <--- HERE
    _10 = (self).post_process_distribution(mean, scale, )
    mean0, std, = _10
  File "code/__torch__/rave/model.py", line 11, in forward
    x: Tensor) -> List[Tensor]:
    net = self.net
    z = (net).forward(x, )
         ~~~~~~~~~~~~ <--- HERE
    _0 = torch.floordiv((torch.size(z))[1], 2)
    return torch.split(z, _0, 1)
  File "code/__torch__/cached_conv/convs.py", line 106, in forward
    _13 = getattr(self, "13")
    _14 = getattr(self, "14")
    input0 = (_0).forward(input, )
              ~~~~~~~~~~~ <--- HERE
    input1 = (_1).forward(input0, )
    input2 = (_2).forward(input1, )
  File "code/__torch__/cached_conv/convs/___torch_mangle_1.py",

In [8]:
output_file = 'output.wav'
torchaudio.save(output_file, waveform_tensor, sr)

audio_display = ipd.Audio(output_file)
display(audio_display)